### Maria Ana Guevarra

# Exercise 3 - Part 2

#### Load the news groups text dataset with the categories below. Create a notebook that performs document classification that assigned the documents to the right new section.

Categories:
1.	alt.atheism
2.	comp.graphics
3.	rec.motorcycles
4.	rec.sport.hockey
5.	sci.electronics
6.	soc.religion.christian
7.	talk.politics.mideast


In [1]:
import nltk
nltk.download('book', quiet=True)

True

In [2]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
)

In [3]:
categories = [
    'alt.atheism',
    'comp.graphics',
    'rec.motorcycles',
    'rec.sport.hockey',
    'sci.electronics',
    'soc.religion.christian',
    'talk.politics.mideast'
]

In [4]:
news = fetch_20newsgroups(subset='test', categories=categories)

In [5]:
documents = news['data']
labels = news['target']

len(documents)

2672

### 1.	Make sure that the data is balanced. One label should not be x1.5 the other one. 


In [6]:
np.unique(labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([319, 389, 398, 399, 393, 398, 376], dtype=int64))

### 2.	Do word tokenization, case folding, and lemmatization on the document

In [7]:
tokenized_cf_docs = [nltk.word_tokenize(d.lower()) for d in documents]

In [8]:
lemmatizer = nltk.WordNetLemmatizer()
Lemmatized_docs = [lemmatizer.lemmatize(d) for d in documents]

### 3.	Split the dataset to 75% 10%, 15% for train, validation, and test respectively.

In [10]:
train_size = 0.75

x_train, x_rem, y_train, y_rem = train_test_split(tokenized_cf_docs, labels, train_size=0.75)

test_size = 3/5

x_valid,x_test,y_valid,y_test = train_test_split(x_rem,y_rem, test_size=3/5)

len(x_train), len(x_valid), len(x_test)

(2004, 267, 401)

### 4.	Create a TF-IDF vector for the tokenized document

In [11]:
cv = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    max_features=2000,
)

cv.fit(x_train)

C:\Users\Ana\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(max_features=2000,
                preprocessor=<function <lambda> at 0x0000017166F67A60>,
                tokenizer=<function <lambda> at 0x0000017166F67940>)

In [12]:
cv.vocabulary_

{'from': 755,
 ':': 98,
 '@': 104,
 '(': 15,
 'l': 999,
 ')': 16,
 'subject': 1704,
 're': 1453,
 '-': 20,
 'tell': 1741,
 'us': 1853,
 'about': 121,
 'it': 948,
 'nntp-posting-host': 1217,
 'reply-to': 1493,
 'organization': 1272,
 'in': 903,
 'the': 1758,
 'hall': 825,
 'lines': 1042,
 '31': 73,
 'mark': 1097,
 'writes': 1969,
 '>': 102,
 'problem': 1398,
 'is': 939,
 'of': 1244,
 'their': 1759,
 'and': 186,
 '.': 25,
 'will': 1939,
 'try': 1820,
 'to': 1792,
 'away': 265,
 'nhl': 1212,
 'espn': 650,
 'also': 172,
 ',': 19,
 'have': 840,
 'control': 478,
 'a': 116,
 'large': 1008,
 'number': 1232,
 'cable': 367,
 'systems': 1723,
 'around': 238,
 'country': 492,
 'with': 1946,
 'total': 1807,
 '15': 41,
 'million': 1135,
 'do': 581,
 'best': 304,
 'that': 1757,
 'never': 1207,
 'gets': 782,
 'off': 1245,
 'ground': 813,
 "'s": 12,
 'value': 1867,
 'this': 1774,
 'war': 1901,
 'between': 306,
 'more': 1160,
 'since': 1622,
 'they': 1768,
 'no': 1218,
 'national': 1188,
 'major': 1084,

In [13]:
train_cvectors = cv.transform(x_train)
val_cvectors = cv.transform(x_valid)
test_cvectors = cv.transform(x_test)

In [14]:
train_cvectors.toarray()

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
tfidf = TfidfTransformer()
tfidf.fit(train_cvectors)

TfidfTransformer()

In [16]:
train_vectors = tfidf.transform(train_cvectors)
val_vectors = tfidf.transform(val_cvectors)
test_vectors = tfidf.transform(test_cvectors)

In [17]:
train_vectors.toarray()

array([[0.1612197 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15331562, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01278802, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### 5.	Use Naïve Bayes to do the classification taking note of the types of implementation below:
##### a.	BernouliNB: Binary/Flags
##### b.	MultinomialNB: Counting Data
##### c.	GaussianNB: Continuous Data


In [18]:
nb = BernoulliNB()
nb.fit(train_vectors, y_train)

BernoulliNB()

In [19]:
train_predict = nb.predict(train_vectors)
val_predict = nb.predict(val_vectors)
test_predict =  nb.predict(test_vectors)

### 6.	Calculate the precision, recall, accuracy, and confusion matrix for each dataset split

### Training Performance

In [20]:
print(classification_report(y_train, train_predict))

              precision    recall  f1-score   support

           0       0.90      0.77      0.83       243
           1       0.83      0.88      0.86       302
           2       0.80      0.97      0.88       289
           3       0.98      0.94      0.96       298
           4       0.69      0.93      0.80       291
           5       0.95      0.73      0.83       292
           6       0.96      0.74      0.84       289

    accuracy                           0.85      2004
   macro avg       0.87      0.85      0.85      2004
weighted avg       0.87      0.85      0.86      2004



In [21]:
confusion_matrix(y_train, train_predict)

array([[186,   4,  30,   1,  12,   5,   5],
       [  5, 267,   7,   0,  21,   1,   1],
       [  0,   4, 280,   0,   4,   0,   1],
       [  2,   8,   1, 281,   3,   3,   0],
       [  0,  17,   1,   0, 272,   1,   0],
       [  3,  16,   9,   2,  47, 214,   1],
       [ 10,   4,  23,   4,  33,   2, 213]], dtype=int64)

### Validation Performance

In [22]:
print(classification_report(y_valid, val_predict))

              precision    recall  f1-score   support

           0       0.88      0.61      0.72        36
           1       0.77      0.80      0.79        30
           2       0.67      0.93      0.78        42
           3       1.00      0.95      0.97        38
           4       0.68      0.86      0.76        37
           5       0.92      0.85      0.88        40
           6       0.94      0.70      0.81        44

    accuracy                           0.82       267
   macro avg       0.84      0.82      0.82       267
weighted avg       0.84      0.82      0.82       267



In [23]:
confusion_matrix(y_valid, val_predict)

array([[22,  0,  8,  0,  2,  2,  2],
       [ 0, 24,  2,  0,  4,  0,  0],
       [ 0,  1, 39,  0,  2,  0,  0],
       [ 0,  0,  0, 36,  2,  0,  0],
       [ 1,  4,  0,  0, 32,  0,  0],
       [ 1,  1,  3,  0,  1, 34,  0],
       [ 1,  1,  6,  0,  4,  1, 31]], dtype=int64)

### Test Performance

In [24]:
print(classification_report(y_test, test_predict))

              precision    recall  f1-score   support

           0       0.76      0.62      0.68        40
           1       0.82      0.88      0.85        57
           2       0.75      0.87      0.81        67
           3       0.97      0.92      0.94        63
           4       0.65      0.86      0.74        65
           5       0.98      0.83      0.90        66
           6       0.93      0.60      0.73        43

    accuracy                           0.82       401
   macro avg       0.84      0.80      0.81       401
weighted avg       0.84      0.82      0.82       401



In [26]:
confusion_matrix(y_test, test_predict)

array([[25,  2,  5,  0,  7,  1,  0],
       [ 1, 50,  1,  0,  5,  0,  0],
       [ 2,  2, 58,  0,  5,  0,  0],
       [ 1,  0,  0, 58,  4,  0,  0],
       [ 1,  5,  2,  0, 56,  0,  1],
       [ 1,  0,  4,  1,  4, 55,  1],
       [ 2,  2,  7,  1,  5,  0, 26]], dtype=int64)

### 7.	Show the best features for each label

In [43]:
best0 = nb.feature_log_prob_[0].argsort()
best1 = nb.feature_log_prob_[1].argsort()
best2 = nb.feature_log_prob_[2].argsort()
best3 = nb.feature_log_prob_[3].argsort()
best4 = nb.feature_log_prob_[4].argsort()
best5 = nb.feature_log_prob_[5].argsort()
best6 = nb.feature_log_prob_[6].argsort()

In [44]:
vocabulary = sorted(cv.vocabulary_)

In [45]:
b0 = [vocabulary[idx] for idx in best0[-10:][::-1]]
b0

[':', 'subject', 'lines', 'from', '@', ',', '.', ')', '(', 'to']

In [46]:
b1 = [vocabulary[idx] for idx in best1[-10:][::-1]]
b1

[':', 'lines', '@', 'from', 'subject', 'organization', ',', '.', ')', '(']

In [47]:
b2 = [vocabulary[idx] for idx in best2[-10:][::-1]]
b2

[':', 'subject', '@', 'from', ',', 'lines', 'organization', '(', ')', '.']

In [48]:
b3 = [vocabulary[idx] for idx in best3[-10:][::-1]]
b3

['lines', '@', 'subject', 'from', ':', 'organization', '.', ',', 'the', ')']

In [49]:
b5 = [vocabulary[idx] for idx in best5[-10:][::-1]]
b5

['subject', ':', 'from', 'lines', '@', '.', '(', ')', ',', 'the']

In [50]:
b6 = [vocabulary[idx] for idx in best6[-10:][::-1]]
b6

[':', 'subject', '@', 'from', 'lines', '(', ')', '.', ',', 'the']

### 8.	Do steps 1-6 using bag of words instead of TF-IDF. Which implementation produced the better result?

In [27]:
np.unique(labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([319, 389, 398, 399, 393, 398, 376], dtype=int64))

In [28]:
tokenized_cf_docs = [nltk.word_tokenize(d.lower()) for d in documents]

In [29]:
lemmatizer = nltk.WordNetLemmatizer()
Lemmatized_docs = [lemmatizer.lemmatize(d) for d in documents]

In [30]:
train_size = 0.75

x_train, x_rem, y_train, y_rem = train_test_split(tokenized_cf_docs, labels, train_size=0.75)

test_size = 3/5

x_valid,x_test,y_valid,y_test = train_test_split(x_rem,y_rem, test_size=3/5)

len(x_train), len(x_valid), len(x_test)

(2004, 267, 401)

In [31]:
cv = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    max_features=2000,
)

cv.fit(x_train)

C:\Users\Ana\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(max_features=2000,
                preprocessor=<function <lambda> at 0x000001715C0FBCA0>,
                tokenizer=<function <lambda> at 0x000001715C0FB820>)

In [32]:
cv.vocabulary_

{'organization': 1275,
 ':': 99,
 'university': 1848,
 'of': 1246,
 'system': 1728,
 'from': 756,
 'the': 1763,
 'always': 179,
 'patrick': 1321,
 '<': 101,
 '@': 105,
 '>': 103,
 'subject': 1709,
 're': 1460,
 '1993': 51,
 'nhl': 1214,
 'draft': 601,
 'lines': 1048,
 '19': 45,
 'san': 1553,
 'will': 1940,
 'then': 1768,
 'get': 781,
 'know': 996,
 'that': 1762,
 'kind': 991,
 'cause': 389,
 'i': 890,
 "'ve": 13,
 'seen': 1581,
 'on': 1261,
 'with': 1948,
 'a': 117,
 'sharks': 1607,
 'pat': 1320,
 '.': 25,
 'go': 791,
 'bruins': 353,
 'black': 322,
 'number': 1234,
 '1': 32,
 'hockey': 869,
 'east': 617,
 'season': 1572,
 '.....': 29,
 'paul': 1322,
 ',': 19,
 '!': 0,
 '(': 15,
 ')': 16,
 'mind': 1144,
 '-': 20,
 'british': 349,
 'canada': 378,
 '13': 39,
 'live': 1052,
 'up': 1853,
 'in': 905,
 'cable': 369,
 'company': 457,
 'use': 1857,
 'is': 941,
 'called': 372,
 'does': 587,
 'anyone': 204,
 'their': 1764,
 'and': 190,
 'ways': 1912,
 'getting': 783,
 'around': 240,
 'them': 1766

In [33]:
train_cvectors = cv.transform(x_train)
val_cvectors = cv.transform(x_valid)
test_cvectors = cv.transform(x_test)

In [34]:
train_cvectors.toarray()

array([[20,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  8,  0,  0],
       [ 7,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [35]:
nb = BernoulliNB()
nb.fit(train_cvectors, y_train)

BernoulliNB()

In [36]:
train_predict = nb.predict(train_cvectors)
val_predict = nb.predict(val_cvectors)
test_predict =  nb.predict(test_cvectors)

In [37]:
print(classification_report(y_train, train_predict))

              precision    recall  f1-score   support

           0       0.91      0.78      0.84       249
           1       0.85      0.90      0.87       287
           2       0.73      0.99      0.84       309
           3       0.98      0.95      0.96       282
           4       0.79      0.93      0.86       298
           5       0.96      0.76      0.85       294
           6       0.98      0.73      0.84       285

    accuracy                           0.86      2004
   macro avg       0.88      0.86      0.86      2004
weighted avg       0.88      0.86      0.86      2004



In [38]:
confusion_matrix(y_train, train_predict)

array([[194,   4,  37,   1,   6,   3,   4],
       [  6, 257,   8,   0,  14,   2,   0],
       [  2,   2, 305,   0,   0,   0,   0],
       [  1,   4,   3, 267,   4,   3,   0],
       [  0,  19,   1,   0, 278,   0,   0],
       [  4,  11,  21,   2,  32, 223,   1],
       [  7,   7,  43,   3,  16,   1, 208]], dtype=int64)

In [39]:
print(classification_report(y_valid, val_predict))

              precision    recall  f1-score   support

           0       0.91      0.69      0.78        29
           1       0.74      0.67      0.70        39
           2       0.67      0.93      0.78        40
           3       1.00      0.95      0.98        42
           4       0.74      0.91      0.82        46
           5       0.97      0.82      0.89        34
           6       0.97      0.76      0.85        37

    accuracy                           0.83       267
   macro avg       0.86      0.82      0.83       267
weighted avg       0.85      0.83      0.83       267



In [40]:
confusion_matrix(y_valid, val_predict)

array([[20,  1,  6,  0,  1,  0,  1],
       [ 0, 26,  4,  0,  9,  0,  0],
       [ 0,  1, 37,  0,  2,  0,  0],
       [ 0,  0,  1, 40,  0,  1,  0],
       [ 0,  4,  0,  0, 42,  0,  0],
       [ 0,  2,  2,  0,  2, 28,  0],
       [ 2,  1,  5,  0,  1,  0, 28]], dtype=int64)

In [41]:
print(classification_report(y_test, test_predict))

              precision    recall  f1-score   support

           0       0.72      0.63      0.68        41
           1       0.79      0.84      0.82        63
           2       0.70      0.94      0.80        49
           3       0.99      0.93      0.96        75
           4       0.68      0.86      0.76        49
           5       0.90      0.67      0.77        70
           6       0.85      0.74      0.79        54

    accuracy                           0.81       401
   macro avg       0.80      0.80      0.80       401
weighted avg       0.82      0.81      0.81       401



In [42]:
print(classification_report(y_test, test_predict))

              precision    recall  f1-score   support

           0       0.72      0.63      0.68        41
           1       0.79      0.84      0.82        63
           2       0.70      0.94      0.80        49
           3       0.99      0.93      0.96        75
           4       0.68      0.86      0.76        49
           5       0.90      0.67      0.77        70
           6       0.85      0.74      0.79        54

    accuracy                           0.81       401
   macro avg       0.80      0.80      0.80       401
weighted avg       0.82      0.81      0.81       401



### Based on the data shown, TF-IDF produces better result. The precision, recall, and accuracy in TF-IDF is greater than in Bag of words.